### Field report: PNL and hit rate analysis for 5 basic strategies.
### 17/03/17- v0.1
#### por mrg

In [130]:
#Load packages
Pkg.add("TimeSeries");Pkg.add("MarketData");Pkg.add("Plots");Pkg.add("PlotlyJS");
Pkg.add("Gadfly");Pkg.add("StatsBase");Pkg.update();

INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: Nothing to be done
INFO: No packages to install, update or remove
INFO: Package database updated
INFO: Updating METADATA...
INFO: Updating cache of DataFrames...
INFO: Updating cache of DataFrames...
INFO: Updating OhMyREPL master...
INFO: Updating Atom master...
INFO: Updating Tokenize master...
INFO: Computing changes...
INFO: No packages to install, update or remove


LoadError: Update finished with errors.
=> Package DataFrames: unable to update cache.
GitError(Code:ERROR, Class:OS, Failed to receive response: The server returned an invalid or unrecognized response
)
 in macro expansion at .\libgit2\error.jl:99 [inlined]
 in #fetch#55(::Base.LibGit2.FetchOptions, ::String, ::Function, ::Base.LibGit2.GitRemote, ::Array{AbstractString,1}) at .\libgit2\remote.jl:70
 in (::Base.LibGit2.#kw##fetch)(::Array{Any,1}, ::Base.LibGit2.#fetch, ::Base.LibGit2.GitRemote, ::Array{AbstractString,1}) at .\<missing>:0
 in #fetch#96(::String, ::String, ::Array{AbstractString,1}, ::Nullable{Base.LibGit2.AbstractCredentials}, ::Function, ::Base.LibGit2.GitRepo) at .\libgit2\libgit2.jl:164
 in fetch(::Base.LibGit2.GitRepo) at .\libgit2\libgit2.jl:156
 in prefetch(::String, ::String, ::Array{String,1}) at .\pkg\cache.jl:64
 in update(::String, ::Set{String}) at .\pkg\entry.jl:403
 in (::Base.Pkg.Dir.##2#3{Array{Any,1},Base.Pkg.Entry.#update,Tuple{String,Set{String}}})()

In [155]:
Pkg.add("PlotRecipes")

INFO: Cloning cache of NetworkLayout from https://github.com/JuliaGraphs/NetworkLayout.jl.git
INFO: Cloning cache of PlotRecipes from https://github.com/JuliaPlots/PlotRecipes.jl.git
INFO: Cloning cache of StatPlots from https://github.com/JuliaPlots/StatPlots.jl.git
INFO: Installing NetworkLayout v0.0.1
INFO: Installing PlotRecipes v0.2.0
INFO: Installing StatPlots v0.2.1
INFO: Building Plots
INFO: Building Rmath
INFO: Package database updated


In [156]:
using TimeSeries, MarketData, Plots, PlotlyJS, StatsBase,PlotRecipes;

INFO: Recompiling stale cache file C:\Users\miguel\.julia\lib\v0.5\Distributions.ji for module Distributions.
INFO: Recompiling stale cache file C:\Users\miguel\.julia\lib\v0.5\DataFrames.ji for module DataFrames.
INFO: Precompiling module KernelDensity.
INFO: Precompiling module GeometryTypes.
  likely near C:\Users\miguel\.julia\v0.5\GeometryTypes\src\typealias.jl:90
INFO: Recompiling stale cache file C:\Users\miguel\.julia\lib\v0.5\LightGraphs.ji for module LightGraphs.


In [131]:
typeof(ohlc)

TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}}

In [132]:
ohlc

500x4 TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}} 2000-01-03 to 2001-12-31

             Open      High      Low       Close     
2000-01-03 | 104.88    112.5     101.69    111.94    
2000-01-04 | 108.25    110.62    101.19    102.5     
2000-01-05 | 103.75    110.56    103.0     104.0     
2000-01-06 | 106.12    107.0     95.0      95.0      
⋮
2001-12-26 | 21.35     22.3      21.14     21.49     
2001-12-27 | 21.58     22.25     21.58     22.07     
2001-12-28 | 21.97     23.0      21.96     22.43     
2001-12-31 | 22.51     22.66     21.83     21.9      


In [133]:
ohlc["Open"]

500x1 TimeSeries.TimeArray{Float64,1,Date,Array{Float64,1}} 2000-01-03 to 2001-12-31

             Open      
2000-01-03 | 104.88    
2000-01-04 | 108.25    
2000-01-05 | 103.75    
2000-01-06 | 106.12    
⋮
2001-12-26 | 21.35     
2001-12-27 | 21.58     
2001-12-28 | 21.97     
2001-12-31 | 22.51     


In [134]:
import TimeSeries.moving

In [135]:
# Función ventana
function moving{T}(ta::TimeArray{T,1}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end])
    @inbounds for i=1:length(vals)
        vals[i] = f(ta.values[i:i+(window-1)])
    end
    padding && (vals = [NaN*ones(window-1); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

function moving{T}(ta::TimeArray{T,2}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end, :])
    @inbounds for i=1:size(vals,1), j=1:size(vals, 2)
        vals[i, j] = f(ta.values[i:i+(window-1), j])
    end
    padding && (vals = [NaN*ones(ta.values[1:(window-1), :]); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

moving (generic function with 2 methods)

In [136]:
#Nice, ya podemos tomar promedios en ventanas de tamaño N = 10
moving(ohlc,mean,10)

D<:Base.Dates.TimeType, A<:AbstractArray}, Function, Int64) in module Main at In[128]:3 overwritten at In[135]:3.

491x4 TimeSeries.TimeArray{Float64,2,Date,Array{Float64,2}} 2000-01-14 to 2001-12-31

             Open      High      Low       Close     
2000-01-14 | 100.692   103.981   96.001    98.782    
2000-01-18 | 100.304   103.331   95.876    97.982    
2000-01-19 | 100.041   103.144   96.095    98.388    
2000-01-20 | 101.216   104.238   97.145    99.338    
⋮
2001-12-26 | 21.062    21.507    20.618    21.065    
2001-12-27 | 21.033    21.54     20.651    21.123    
2001-12-28 | 21.081    21.685    20.797    21.266    
2001-12-31 | 21.259    21.868    20.971    21.417    


In [139]:
# Aquí se van a desarrollar las funciones estrategia

In [144]:
moving(ohlc,psnr,10)

LoadError: MethodError: no method matching psnr(::Array{Float64,1})[0m
Closest candidates are:
  psnr{T<:Real}(::AbstractArray{T<:Real,N}, [1m[31m::AbstractArray{T<:Real,N}[0m, [1m[31m::Real[0m) at C:\Users\miguel\.julia\v0.5\StatsBase\src\deviation.jl:183[0m

In [145]:
?psnr

search: psnr specialized_bitwise_unary specialized_bitwise_binary flipsign



```
psnr(a, b, maxv)
```

Compute the peak signal-to-noise ratio between two arrays `a` and `b`. `maxv` is the maximum possible value either array can take. The PSNR is computed as `10 * log10(maxv^2 / msd(a, b))`.


In [149]:
using TimeSeries

In [150]:
@recipe function f{T<:TimeArray}(ta::T)
    st = get(d, :seriestype, :path)
    if in(st, [:candlestick, :heikinashi])
        Candlestick(ta)
    #elseif st == :ohlc #ohlc (meaning sticks with steps on the sides) should be passed on to Plots internal ohlc plot engine
    #    ta, ohlc = extract_ohlc(ta)
    #    collect(zip(ohlc)) # But there are currently issues with that
    else
        labels --> reshape(ta.colnames,1,length(ta.colnames))
        seriestype := st
        ta.timestamp, ta.values
    end
end

type Candlestick{D <: TimeType}
    time::Vector{D}
    open::AbstractVector
    high::AbstractVector
    low::AbstractVector
    close::AbstractVector
end

Candlestick(ta::TimeArray) = Candlestick(extract_ohlc(ta)...)

function extract_ohlc(ta::TimeArray)
    indices = [find(x->lowercase(x) == name, ta.colnames) for name in ["open", "high", "low", "close"]]
    minimum(length.(indices)) < 1 && error("The time array did not have variables named open, high, low and close")
    (ta.timestamp, [ta.values[:,i] for i in 1:4]...)
end

function HeikinAshi!(cs::Candlestick) #some values here are made too high!
    cs.close[1] = (cs.open[1] + cs.low[1] + cs.close[1] + cs.high[1]) / 4
    cs.open[1] = (cs.open[1] + cs.close[1])/2
    cs.high[1] = cs.high[1]
    cs.low[1] = cs.low[1]

    for i in 2:length(cs.open)
        cs.close[i] = (cs.open[i] + cs.low[i] + cs.close[i] + cs.high[i]) / 4
        cs.open[i] = (cs.open[i-1] + cs.close[i-1]) / 2
        cs.high[i] = maximum([cs.high[i], cs.open[i], cs.close[i]])
        cs.low[i] = minimum([cs.low[i], cs.open[i], cs.close[i]])
    end
end

@recipe function f(cs::Candlestick)
    st = get(d, :seriestype, :candlestick)
    st == :heikinashi && HeikinAshi!(cs)

    seriestype := :candlestick
    legend --> false
    linewidth --> 0.7
    grid --> false

    bw = get(d, :bar_width, nothing)
    bw == nothing && (bw = 0.8)
    bar_width := bw / 2 * minimum(diff(unique(Int.(cs.time))))

    # allow passing alternative colors as a vector
    cols = get(d, :seriescolor, nothing)
    cols = (isa(cols, Vector{Symbol}) && length(cols) == 2) ? cols : [:red, :blue]

    attributes = [
        Dict(:close_open => <, :close_prev => <, :bottombox => cs.close, :topbox => cs.open, :fill => cols[1], :line => cols[1], :fillalpha => 1),
        Dict(:close_open => <, :close_prev => >=, :bottombox => cs.close, :topbox => cs.open, :fill => cols[2], :line => cols[2], :fillalpha => 1),
        Dict(:close_open => >=, :close_prev => <, :bottombox => cs.open, :topbox => cs.close, :fill => :white, :line => cols[1], :fillalpha => 0),
        Dict(:close_open => >=, :close_prev => >=, :bottombox => cs.open, :topbox => cs.close, :fill => :white, :line => cols[2], :fillalpha => 0)
    ]


    for att in attributes
        inds = Vector{Int}(length(cs.close))
        inds[1] = att[:close_open](cs.close[1], cs.open[1]) & att[:close_prev](cs.close[1], cs.close[1])
        inds[2:end] .= att[:close_open].(cs.close[2:end], cs.open[2:end]) & att[:close_prev].(diff(cs.close), 0)
        inds = find(inds)

        if length(inds) > 0
            @series begin
                linecolor := att[:line]
                fillcolor := att[:fill]
                fillalpha := att[:fillalpha]
                fillto := att[:bottombox][inds]
                seriestype := :bar
                cs.time[inds], att[:topbox][inds]
            end

            for j in 1:2
                @series begin
                    primary := false
                    linecolor := att[:line]
                    seriestype := :sticks
                    fillto := j == 1 ? cs.low[inds] : att[:topbox][inds]
                    cs.time[inds], j == 1 ? att[:bottombox][inds] : cs.high[inds]
                end
            end
        end
    end
end

In [154]:
@recipe Candlestick(ohlc)

LoadError: Must wrap a valid function call!

In [152]:
plot(Candlestick(ohlc))

In [153]:
@recipe function f{T<:TimeArray}(ta::T)
    st = get(d, :seriestype, :path)
    if in(st, [:candlestick, :heikinashi])
        Candlestick(ta)
    #elseif st == :ohlc #ohlc (meaning sticks with steps on the sides) should be passed on to Plots internal ohlc plot engine
    #    ta, ohlc = extract_ohlc(ta)
    #    collect(zip(ohlc)) # But there are currently issues with that
    else
        labels --> reshape(ta.colnames,1,length(ta.colnames))
        seriestype := st
        ta.timestamp, ta.values
    end
end

In [157]:
Pkg.add("Indicators")

INFO: Cloning cache of Indicators from https://github.com/dysonance/Indicators.jl.git
INFO: Cloning cache of Temporal from https://github.com/dysonance/Temporal.jl.git
INFO: Installing Indicators v0.1.4
INFO: Installing Temporal v0.2.4
INFO: Building HttpParser
INFO: Building MbedTLS


Using system libraries...


INFO: Package database updated


In [158]:
using Indicators

INFO: Precompiling module Indicators.


In [161]:
ohlc["Open"]

500x1 TimeSeries.TimeArray{Float64,1,Date,Array{Float64,1}} 2000-01-03 to 2001-12-31

             Open      
2000-01-03 | 104.88    
2000-01-04 | 108.25    
2000-01-05 | 103.75    
2000-01-06 | 106.12    
⋮
2001-12-26 | 21.35     
2001-12-27 | 21.58     
2001-12-28 | 21.97     
2001-12-31 | 22.51     


In [163]:
Indicators.

Indicators

In [164]:
function module_functions(modname)
    list = Symbol[]
    for nm in names(modname)
        typeof(eval(nm)) == Function && push!(list,nm)
    end
    return list
end
module_functions(Indicators)

module_functions (generic function with 1 method)

In [165]:
module_functions(Indicators)

0-element Array{Symbol,1}

In [170]:
Indicators.adx
Indicators.alma
Indicators.aroon
Indicators.atr
Indicators.bbands
Indicators.cci
Indicators.close_fun
Indicators.crossover
Indicators.crossunder
Indicators.dema
Indicators.diffn
Indicators.donch
Indicators.ema
Indicators.eval
Indicators.hl_fun
Indicators.hlc_fun
Indicators.hma
Indicators.interpolate
Indicators.kama
Indicators.keltner
Indicators.kst
Indicators.macd
Indicators.mama
Indicators.maxima
Indicators.minima
Indicators.mlr
Indicators.mlr_bands
Indicators.mlr_intercept
Indicators.mlr_lb
Indicators.mlr_rsq
Indicators.mlr_se
Indicators.mlr_slope
Indicators.mlr_ub
Indicators.psar
Indicators.resistance
Indicators.roc
Indicators.rsi
Indicators.runcor
Indicators.runcov
Indicators.runmad
Indicators.runmax
Indicators.runmean
Indicators.runsd
Indicators.runsum
Indicators.runvar
Indicators.sma
Indicators.smi
Indicators.stoch
Indicators.support
Indicators.swma
Indicators.tema
Indicators.tr
Indicators.trima
Indicators.wilder_sum
Indicators.wma
Indicators.wpr






LoadError: syntax: incomplete: premature end of input

In [166]:
names(Indicators)

55-element Array{Symbol,1}:
 :Indicators
 :adx       
 :alma      
 :aroon     
 :atr       
 :bbands    
 :cci       
 :crossover 
 :crossunder
 :dema      
 :diffn     
 :donch     
 :ema       
 ⋮          
 :runsum    
 :runvar    
 :sma       
 :smi       
 :stoch     
 :swma      
 :tema      
 :tr        
 :trima     
 :wilder_sum
 :wma       
 :wpr       